In [1]:
'xd'

'xd'

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, 
    MobileBertForSequenceClassification, 
    Trainer, 
    TrainingArguments
)

# 1. Wczytanie i przygotowanie danych [cite: 87, 88]
df = pd.read_csv("intents_examples.csv")

# Mapowanie etykiet tekstowych na liczbowe
label_map = {"REPEAT": 0, "NOT_BREATHING": 1}
df['label'] = df['intent'].map(label_map)

# Podział na zbiór treningowy i testowy (np. 80/20) [cite: 88, 90]
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), 
    df['label'].tolist(), 
    test_size=0.2, 
    random_state=42
)

# 2. Tokenizacja [cite: 62]
model_name = "google/mobilebert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# 3. Tworzenie obiektu Dataset dla PyTorch
class IntentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_labels)
val_dataset = IntentDataset(val_encodings, val_labels)

id2label = {0: "REPEAT", 1: "NOT_BREATHING"}
label2id = {"REPEAT": 0, "NOT_BREATHING": 1}

# 4. Inicjalizacja modelu MobileBERT [cite: 82]
model = MobileBertForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=len(label_map),
    id2label=id2label,
    label2id=label2id
)

# 5. Konfiguracja treningu [cite: 136]
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,              # Przy małym zbiorze 5 epok wystarczy
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.1,
    logging_steps=1,
    logging_dir="./logs",
    eval_strategy="epoch",      # Ocena modelu po każdej epoce [cite: 137]
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5               # Mały LR zapobiega przeuczeniu
)

# 6. Uruchomienie Trenera
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# 7. Zapisanie modelu
model.save_pretrained("./fine_tuned_mobilebert")
tokenizer.save_pretrained("./fine_tuned_mobilebert")
print("Model gotowy!")

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,5179283.000000,6394387.500000
2,634845.812500,77780.007812
3,2163.725600,43.342289
4,0.581700,0.520651
5,0.658600,0.511659
6,0.264100,0.156568
7,0.211800,0.103197
8,0.094700,0.100062
9,123.929700,0.037159
10,0.030100,0.038046


Model gotowy!


In [16]:
from transformers import pipeline

# Załadowanie wytrenowanego modelu
pipe = pipeline("text-classification", model="./fine_tuned_mobilebert", tokenizer="./fine_tuned_mobilebert")

# Testowe zdania
test_sentences = [
    "Możesz powtórzyć?",           # Powinno być REPEAT
    "On chyba nie oddycha!",       # Powinno być NOT_BREATHING
    "Jeszcze raz proszę",          # Powinno być REPEAT
    "nie zrozumiałem",
    "nie rozumiem",
    "jeszcze raz"
]

for sentence in test_sentences:
    result = pipe(sentence)
    print(f"Tekst: {sentence} -> Wynik: {result}")

Device set to use cuda:0


Tekst: Możesz powtórzyć? -> Wynik: [{'label': 'REPEAT', 'score': 0.9993758797645569}]
Tekst: On chyba nie oddycha! -> Wynik: [{'label': 'NOT_BREATHING', 'score': 0.9968045949935913}]
Tekst: Jeszcze raz proszę -> Wynik: [{'label': 'REPEAT', 'score': 0.995864987373352}]
Tekst: nie zrozumiałem -> Wynik: [{'label': 'REPEAT', 'score': 0.9993590712547302}]
Tekst: nie rozumiem -> Wynik: [{'label': 'REPEAT', 'score': 0.5444620847702026}]
Tekst: jeszcze raz -> Wynik: [{'label': 'REPEAT', 'score': 0.9967953562736511}]


In [1]:
'test'

'test'

In [2]:
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sentence_transformers import losses

In [1]:
import pandas as pd
import yaml
import re
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split
from sentence_transformers import losses
from sklearn.metrics import classification_report

# --- FUNKCJA WCZYTUJĄCA YAML ---
def load_nlu_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
    
    rows = []
    for item in data['nlu']:
        intent = item['intent']
        examples = item['examples'].strip().split('\n')
        for ex in examples:
            clean_text = re.sub(r'^-\s*', '', ex).strip().strip('"')
            if clean_text:
                rows.append({"intent": intent, "text": clean_text})
    return pd.DataFrame(rows)

# 1. Wczytanie danych
df = load_nlu_data("first-aid-assistant/nlu.yml")

# --- DYNAMICZNE TWORZENIE LABEL_MAP ---
# Pobieramy unikalne intencje i przypisujemy im numery
unique_intents = sorted(df['intent'].unique())
label_map = {intent: i for i, intent in enumerate(unique_intents)}

print(f"Wykryto {len(label_map)} intencji.")
df['label'] = df['intent'].map(label_map)

# Podział na zbiory (stratify zapewnia równe rozłożenie rzadkich intencji)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# 2. Inicjalizacja modelu
# labels_list musi być w kolejności indeksów (0, 1, 2...)
labels_list = unique_intents 

model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    labels=labels_list
)

# 4. Inicjalizacja Trenera
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=losses.CosineSimilarityLoss,
    metric="accuracy",
    column_mapping={"text": "text", "label": "label"},
    num_epochs=1,
    batch_size=16,
    use_amp=True
)

# 5. Trening
trainer.train()

# 6. Ewaluacja
metrics = trainer.evaluate()
print(f"\nOgólne Accuracy: {metrics['accuracy']:.4f}")

y_true = test_df['intent'].values 
y_pred = model.predict(test_df['text'].tolist())

print("\nSzczegółowy raport klasyfikacji:")
print("-" * 60)
print(classification_report(y_true, y_pred))

# 7. Zapisanie modelu
model.save_pretrained("./setfit_intent_model")

Wykryto 20 intencji.


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\mkowa\AppData\Local\Temp\ipykernel_28976\1451016425.py:51: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/219 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8760
  Batch size = 16
  Num epochs = 1
c:\Dev\first-aid-assistant-app-nlu-model\.venv\Lib\site-packages\torch\utils\data\_utils\pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.pin_memory() is deprecated. Please do not pass this argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Memory.cpp:48.)
  return data.pin_memory(device)
c:\Dev\first-aid-assistant-app-nlu-model\.venv\Lib\site-packages\torch\utils\data\_utils\pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.is_pinned() is deprecated. Please do not pass this argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Memory.cpp:33.)
  return data.pin_memory(device)


Step,Training Loss
1,0.164900
50,0.147900
100,0.110000
150,0.064600
200,0.042700
250,0.039900
300,0.033100
350,0.026100
400,0.024700
450,0.019000


c:\Dev\first-aid-assistant-app-nlu-model\.venv\Lib\site-packages\torch\utils\data\_utils\pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.pin_memory() is deprecated. Please do not pass this argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Memory.cpp:48.)
  return data.pin_memory(device)
c:\Dev\first-aid-assistant-app-nlu-model\.venv\Lib\site-packages\torch\utils\data\_utils\pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.is_pinned() is deprecated. Please do not pass this argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Memory.cpp:33.)
  return data.pin_memory(device)
***** Running evaluation *****



Ogólne Accuracy: 0.7455

Szczegółowy raport klasyfikacji:
------------------------------------------------------------
                          precision    recall  f1-score   support

        ASSESS_BREATHING       0.50      0.67      0.57         3
    ASSESS_CONSCIOUSNESS       1.00      1.00      1.00         3
           ASSESS_SAFETY       1.00      1.00      1.00         2
               CALL_HELP       1.00      1.00      1.00         2
    DONT_KNOW_WHAT_TO_DO       1.00      1.00      1.00         3
            LOCATION_AED       1.00      1.00      1.00         2
PROCEDURE_CHOKING_ACTION       0.50      0.33      0.40         3
           PROCEDURE_CPR       0.00      0.00      0.00         3
     PROCEDURE_CPR_CHILD       1.00      0.50      0.67         2
    PROCEDURE_HEMORRHAGE       1.00      0.33      0.50         3
      PROCEDURE_RECOVERY       0.67      0.67      0.67         3
            QUANTITY_ASK       0.50      1.00      0.67         3
                  REP

In [2]:
import pandas as pd
import yaml
import re
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split
from sentence_transformers import losses
from sklearn.metrics import classification_report
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

tokenizer

BertTokenizerFast(name_or_path='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [2]:
import pandas as pd
from setfit import SetFitModel

# 1. Konfiguracja
THRESHOLD = 0.45  # Jeśli pewność < 45%, bot prosi o doprecyzowanie
MODEL_PATH = "./setfit_intent_model"
TEST_FILE = "intents_test.csv"

# 2. Załadowanie modelu i danych
model = SetFitModel.from_pretrained(MODEL_PATH, fix_mistral_regex=True)
test_df = pd.read_csv(TEST_FILE)

test_sentences = test_df['text'].tolist()
true_intents = test_df['intent'].tolist()

# 3. Wykonanie predykcji
probabilities = model.predict_proba(test_sentences)

print(f"{'WYNIK':<10} | {'PRZEWIDZIANA':<25} | {'RZECZYWISTA':<25} | {'PEWNOŚĆ'}")
print("-" * 110)

correct_count = 0

for i, (sentence, true_label) in enumerate(zip(test_sentences, true_intents)):
    # Pobranie najwyższego prawdopodobieństwa i odpowiadającej mu etykiety
    probs = probabilities[i]
    max_idx = probs.argmax().item()
    score = probs[max_idx].item()
    pred_label = model.labels[max_idx]
    
    # Logika Threshold - jeśli zbyt niska pewność, oznacz jako LOW_CONFIDENCE
    final_label = pred_label if score >= THRESHOLD else "LOW_CONFIDENCE"
    
    # Sprawdzenie poprawności (czy przewidziana intencja zgadza się z plikiem testowym)
    is_correct = (final_label == true_label)
    status = "OK" if is_correct else "BŁĄD"
    if is_correct: correct_count += 1
    
    # Wyświetlanie wyników z kolorem (opcjonalnie w terminalu)
    print(f"{status:<10} | {final_label:<25} | {true_label:<25} | {score:.2%} | {sentence}")

# 4. Podsumowanie statystyczne
accuracy = (correct_count / len(test_df)) * 100
print("-" * 110)
print(f"Ogólna dokładność (z Threshold {THRESHOLD:.0%}): {accuracy:.2f}%")

The tokenizer you are loading from './setfit_intent_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


WYNIK      | PRZEWIDZIANA              | RZECZYWISTA               | PEWNOŚĆ
--------------------------------------------------------------------------------------------------------------
OK         | SITUATION_UNCONSCIOUS     | SITUATION_UNCONSCIOUS     | 97.84% | Znalazłem kogoś na chodniku, leży nieruchomo.
OK         | SITUATION_UNCONSCIOUS     | SITUATION_UNCONSCIOUS     | 97.04% | Ta osoba leży na ziemii i nie ma z nią kontaktu.
OK         | SITUATION_UNCONSCIOUS     | SITUATION_UNCONSCIOUS     | 88.67% | Chyba ktoś tu zemdlał, bo w ogóle nie reaguje.
OK         | SITUATION_CHOKING         | SITUATION_CHOKING         | 77.93% | On się chyba poważnie zadławił jakimś kawałkiem.
OK         | SITUATION_CHOKING         | SITUATION_CHOKING         | 91.00% | Pomocy, on dusi się i nie może złapać tchu!
OK         | SITUATION_BLEEDING        | SITUATION_BLEEDING        | 98.84% | Z tej rany płynie bardzo duża ilość krwi.
OK         | SITUATION_BLEEDING        | SITUATION_BLEEDING        